**USEFUL LINKS**

DATA DICTIONARY : http://bt.mta.info/wiki/Developers/SIRIMonitoredVehicleJourney

STATIC DATA RESOURCES : http://web.mta.info/developers/developer-data-terms.html#data

DATA FEEDS INFO: https://api.mta.info/#/landing

DEVELOPER RESOURCES: https://new.mta.info/developers

----

In [1]:
import pandas as pd

import os
import requests
import json

In [ ]:
stops = pd.read_csv("./secrets/bus/stops.txt")

In [ ]:
t_stops = stops[stops["stop_name"].str.contains("W 125 ST/MAL")]

In [ ]:
t_stops

In [ ]:
stops = stops[["stop_id", "stop_name"]]

In [ ]:
api_key = "API KEY HERE"

# Load in API key
with open("./secrets/bus_api_key.txt") as f:
    api_key = next(f)
    
stop_id = "402507"

url = f"http://bustime.mta.info/api/siri/stop-monitoring.json?key={api_key}&OperatorRef=MTA&MonitoringRef={stop_id}"

response = requests.request("GET", url)

# Create Python object from JSON string data
obj = json.loads(response.text)
  
# Pretty Print JSON
json_formatted_str = json.dumps(obj, indent=4)

In [ ]:
closest_busses = obj["Siri"]["ServiceDelivery"]["StopMonitoringDelivery"][0]['MonitoredStopVisit'][0:3]

In [ ]:
bus = closest_busses[0]

In [ ]:
# Destination
bus['MonitoredVehicleJourney']['DestinationName']

In [ ]:
temp = pd.DataFrame()

In [ ]:
config_type = ["bus", "subway"]
stop_id = ["402507", "225"]
direction = ["West", "STATION"]

In [ ]:
temp['type'] = config_type
temp['id'] = stop_id
temp['direction'] = direction

In [ ]:
temp.to_csv("./config/config.csv", index=False)

In [ ]:
bus_stops = pd.read_csv("./config/bus_stops.csv")
bus_stops = bus_stops[bus_stops["stop_id"] == int("402507")]
# Return the name of the bus stop
print(bus_stops["stop_name"].tolist()[0])

In [2]:
from google.transit import gtfs_realtime_pb2
from protobuf_to_dict import protobuf_to_dict

In [3]:
subway_api_key = "API KEY HERE"

# Load in API key
with open("./secrets/subway_api_key.txt") as f:
    subway_api_key = next(f)

subway_headers = {"x-api-key": subway_api_key}
subway_url = f"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs"
subway_res = requests.request("GET", subway_url, headers=subway_headers)

In [4]:
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(subway_res.content)

174608

In [5]:
subway_feed = protobuf_to_dict(feed) # subway_feed is a dictionary
realtime_data = subway_feed['entity'] # train_data is a list

In [6]:
import datetime as dt

In [7]:
stop_ids = []
route_ids = []
arrival_times = []
departure_times = []

for train in realtime_data:
    if 'trip_update' in train:
        t_route_id = train['trip_update']['trip']['route_id']
        for stop in train['trip_update']['stop_time_update']:
            if "225" in stop['stop_id']:
                # Grab stop id
                stop_ids.append(stop['stop_id'])
                
                # Grab route id
                route_ids.append(t_route_id)
                
                # Grab arrival time
                
                arrival_times.append(dt.datetime.fromtimestamp(stop['arrival']['time']))
                
                # Grab departure time
                departure_times.append(dt.datetime.fromtimestamp(stop['departure']['time']))
    else:
        pass

In [8]:
df = pd.DataFrame()
df['stop'] = stop_ids
df['route'] = route_ids
df['arrival_time'] = arrival_times
df['departure_time'] = departure_times

In [9]:
df = df.sort_values(by=['arrival_time'])

In [10]:
df

,stop,route,arrival_time,departure_time
0,225N,2,2022-04-26 14:10:49,2022-04-26 14:10:49
24,225S,3,2022-04-26 14:12:59,2022-04-26 14:12:59
18,225N,3,2022-04-26 14:15:29,2022-04-26 14:15:29
3,225S,2,2022-04-26 14:16:00,2022-04-26 14:16:00
26,225S,3,2022-04-26 14:19:30,2022-04-26 14:19:30
1,225N,2,2022-04-26 14:24:24,2022-04-26 14:24:24
19,225N,3,2022-04-26 14:25:59,2022-04-26 14:25:59
28,225S,3,2022-04-26 14:27:30,2022-04-26 14:27:30
5,225S,2,2022-04-26 14:29:42,2022-04-26 14:29:42
2,225N,2,2022-04-26 14:29:45,2022-04-26 14:29:45


In [ ]:
realtime_data[2]['trip_update']['stop_time_update']

In [ ]:
sub_stops = pd.read_csv("./secrets/subway/stops.txt")

In [ ]:
sub_stops.head(1)

In [ ]:
sub_stops = sub_stops[["stop_id", "stop_name"]]

In [ ]:
stop_dir = []

for index, row in sub_stops.iterrows():
    temp = row["stop_id"]
    if "N" in temp[-1]:
        stop_dir.append("UPT")
    elif "S" in temp[-1]:
        stop_dir.append("DWT")
    else:
        stop_dir.append("STATION")

In [ ]:
sub_stops['direction'] = stop_dir

In [ ]:
sub_stops.to_csv("subway_stops.csv", index=False)

In [ ]:
sub_routes = pd.read_csv("./secrets/subway/routes.txt")

In [ ]:
sub_routes = sub_routes[['route_id', 'route_long_name']]

In [ ]:
sub_routes.to_csv("subway_routes.csv", index=False)

In [ ]:
def station_time_lookup(train_data, station):
    for trains in train_data: # trains are dictionaries
        if trains.get('trip_update', False) != False:
            unique_train_schedule = trains['trip_update'] # train_schedule is a dictionary with trip and stop_time_update
            unique_arrival_times = unique_train_schedule['stop_time_update'] # arrival_times is a list of arrivals
            for scheduled_arrivals in unique_arrival_times: #arrivals are dictionaries with time data and stop_ids
                if scheduled_arrivals.get('stop_id', False) == station:
                    time_data = scheduled_arrivals['arrival']
                    unique_time = time_data['time']
                    if unique_time != None:
                        collected_times.append(unique_time)

# Run the above function for the station ID for Broadway-Lafayette
station_time_lookup(realtime_data, 'D21S')